In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def float2pcm(sig,dtype ='int16'):
    sig = np.asarray(sig)
    dtype = np.dtype(dtype)
    i = np.iinfo(dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig * abs_max + offset).clip(i.min, i.max).astype(dtype)

In [15]:
import sounddevice as sd
from scipy.io.wavfile import write
import scipy.io.wavfile as wavfile
import wave
from scipy.fft import rfft, rfftfreq
from pydub import AudioSegment

#sampling frequency
sample_freq = 50000
trail = 1
#recording duration
duration = 5



#Start Recording
background = sd.rec(int(duration * sample_freq ),
                   samplerate = sample_freq, channels= 2)
#record audio for the given number of seconds
print("background recording ...")
sd.wait()
print("background complete")


#Write recording into wav document
write("sound_record\Background.wav",sample_freq,background)

#Actual Recording 
recording = sd.rec(int(duration * sample_freq ),
                   samplerate = sample_freq, channels= 2)

print("Actual recording ...")
sd.wait()
print("Actual complete")

#Write recording into wav document
write("sound_record\sound_recording.wav", sample_freq, recording)

#Convert wav file to a wave module recognizable 

background_alternate = AudioSegment.from_wav("sound_record/Background.wav")
background_alternate.export("sound_record/background_output.wav", format="wav", codec="pcm_s16le")

#Fourier Transform ------------------------------------------------------------

#Background wave open
wave_back = wave.open("sound_record/background_output.wav", 'rb')
n_back = wave_back.getnframes()
t_back = n_back/sample_freq
#Actual Recording wave open
wave_actual = wave.open("sound_record\sound_recording.wav", 'rb')
n_actual = wave_actual.getnframes()
t_actual = n_actual/sample_freq

#background audio process
frames_back = (wave_back.readframes(n_back))
framearray_back = (np.frombuffer(frames_back, dtype=np.int16))
l_channel_back = framearray_back[0::2]
#actual audio
frames_actual = (wave_actual.readframes(n_actual))
framearray_actual = (np.frombuffer(frames_actual, dtype=np.int16))
l_channel_actual = framearray_actual[0::2]

times = np.linspace(0, n_actual/sample_freq, num=n_actual)


plt.figure(figsize=(15, 5))
plt.plot(times, l_channel_back)
plt.plot(times, l_channel_actual)
plt.title('amplitude time plot')
plt.ylabel('amplitude')
plt.xlabel('time in seconds')
plt.xlim(0, t_back)
plt.show()

yf_back = rfft(l_channel_back)
yf_actual = rfft(l_channel_actual)
xf = rfftfreq(n_actual, 1 / sample_freq)
yf = yf_actual - yf_back


plt.plot(xf, np.abs(yf))
plt.title('frequencies and intensity')
plt.xlabel('frequency')
plt.ylabel('intensity')
plt.show()

background recording ...
background complete
Actual recording ...
Actual complete


FileNotFoundError: [WinError 2] 系统找不到指定的文件。

In [5]:
## does fourier transform on an audio file
## has to be WAV
## Code by Bala Venkataraman, MEA CAYPT team

import wave
import numpy as np
from scipy.fft import rfft, rfftfreq

wav_obj = wave.open(input("wav file: "), 'rb')
sample_freq = wav_obj.getframerate()
n_samples = wav_obj.getnframes()
t_audio = n_samples/sample_freq
frames = (wav_obj.readframes(n_samples))
framearray = (np.frombuffer(frames, dtype=np.int16))
l_channel = framearray[0::2]
times = np.linspace(0, n_samples/sample_freq, num=n_samples)

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.plot(times, l_channel)

plt.title('amplitude time plot')
plt.ylabel('amplitude')
plt.xlabel('time in seconds')
plt.xlim(0, t_audio)
plt.show()

yf = rfft(l_channel)
xf = rfftfreq(n_samples, 1 / sample_freq)

plt.plot(xf, np.abs(yf))
plt.title('frequencies and intensity')
plt.xlabel('frequency')
plt.ylabel('intensity')
plt.show()

Error: unknown format: 3